<a href="https://colab.research.google.com/github/tanzinaema/Tanzina_INFO5731_Fall2025/blob/main/For%20Practice/INFO5731_Assignment_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [8]:
!pip install semanticscholar

In [11]:
import pandas as pd
import warnings
from semanticscholar import SemanticScholar
import time

# Ignore warnings
warnings.filterwarnings("ignore")

# Initialize Semantic Scholar client
sem_sch = SemanticScholar()

# Queries
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]

# Collect all results in a list first
papers = []

for query in queries:
    print(f"\nCollecting papers for query: {query}")
    count = 0

    try:
        # Fetch in batches of 100
        for paper in sem_sch.search_paper(
            query,
            limit=100,  # API max
            fields=['paperId', 'title', 'abstract', 'year', 'citationCount']
        ):
            papers.append({
                'query': query,
                'paperId': paper.paperId,
                'title': paper.title,
                'abstract': paper.abstract,
                'year': paper.year,
                'citations': paper.citationCount
            })

            count += 1
            if count >= 2500:  # cap at 2500 per query
                break

        print(f"  → Collected {count} papers for query: {query}")

    except Exception as e:
        print("Error:", e)

# Convert list → DataFrame once
df_paper = pd.DataFrame(papers)

# Save to CSV
df_paper.to_csv("semantic_scholar_top10000.csv", index=False, encoding="utf-8")

print(f"\n✅ Final total: {len(df_paper)} papers saved to semantic_scholar_top10000.csv")



  → Collected 1000 papers for query: machine learning

  → Collected 1000 papers for query: data science

  → Collected 1000 papers for query: artificial intelligence

  → Collected 1000 papers for query: information extraction

✅ Final total: 4000 papers saved to semantic_scholar_top10000.csv


In [12]:
import requests
import pandas as pd
import time

# Define the API endpoint
API_URL = "https://api.semanticscholar.org/graph/v1/paper/search"

# Define the search queries and parameters
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]
papers_per_query = 2500  # 2500 × 4 = 10,000 total
batch_size = 100         # max allowed by API per request

# Store all paper results here
all_papers = []

for query in queries:
    print(f"\n🔍 Collecting top {papers_per_query} papers for query: '{query}'...")
    offset = 0

    while offset < papers_per_query:
        params = {
            "query": query,
            "offset": offset,
            "limit": batch_size,
            "sort": "citationCount",  # highest cited papers first
            "fields": "title,abstract,year,authors,citationCount"
        }

        response = requests.get(API_URL, params=params)
        if response.status_code != 200:
            print(f"❌ Error at offset {offset}: {response.status_code}")
            break

        data = response.json().get("data", [])
        if not data:
            print("⚠️ No more results.")
            break

        for paper in data:
            all_papers.append({
                "query": query,
                "title": paper.get("title"),
                "abstract": paper.get("abstract"),
                "year": paper.get("year"),
                "citations": paper.get("citationCount"),
                "authors": ", ".join([author["name"] for author in paper.get("authors", [])])
            })

        offset += batch_size
        time.sleep(0.2)  # to avoid rate limiting

    print(f"✅ Collected {offset} papers for '{query}'")

# Convert to DataFrame
df = pd.DataFrame(all_papers)

# Save to CSV
df.to_csv("semantic_scholar_top10000.csv", index=False, encoding="utf-8")

# Print first 5 rows
print("\n📄 First 5 papers collected:")
print(df.head())



🔍 Collecting top 2500 papers for query: 'machine learning'...
❌ Error at offset 900: 429
✅ Collected 900 papers for 'machine learning'

🔍 Collecting top 2500 papers for query: 'data science'...
❌ Error at offset 200: 429
✅ Collected 200 papers for 'data science'

🔍 Collecting top 2500 papers for query: 'artificial intelligence'...
❌ Error at offset 200: 429
✅ Collected 200 papers for 'artificial intelligence'

🔍 Collecting top 2500 papers for query: 'information extraction'...
❌ Error at offset 0: 429
✅ Collected 0 papers for 'information extraction'

📄 First 5 papers collected:
              query                                              title  \
0  machine learning  Fashion-MNIST: a Novel Image Dataset for Bench...   
1  machine learning                  Physics-informed machine learning   
2  machine learning  TensorFlow: Large-Scale Machine Learning on He...   
3  machine learning  A Survey on Bias and Fairness in Machine Learning   
4  machine learning  Stop explaining black 

In [13]:
import requests
import pandas as pd
import time

API_URL = "https://api.semanticscholar.org/graph/v1/paper/search"
queries = ["machine learning", "data science", "artificial intelligence", "information extraction"]
papers_per_query = 2500
batch_size = 100

all_papers = []

for query in queries:
    print(f"\n🔍 Collecting top {papers_per_query} papers for query: '{query}'...")
    offset = 0

    while offset < papers_per_query:
        params = {
            "query": query,
            "offset": offset,
            "limit": batch_size,
            "sort": "citationCount",
            "fields": "title,abstract,year,authors,citationCount"
        }

        response = requests.get(API_URL, params=params)

        # Handle rate limiting (HTTP 429)
        if response.status_code == 429:
            print(f"⏳ Hit rate limit at offset {offset}. Sleeping for 60 seconds...")
            time.sleep(60)
            continue  # retry same offset

        # Handle other request errors
        if response.status_code != 200:
            print(f"❌ Error at offset {offset}: {response.status_code}")
            break

        data = response.json().get("data", [])
        if not data:
            print("⚠️ No more results.")
            break

        for paper in data:
            all_papers.append({
                "query": query,
                "title": paper.get("title"),
                "abstract": paper.get("abstract"),
                "year": paper.get("year"),
                "citations": paper.get("citationCount"),
                "authors": ", ".join([author["name"] for author in paper.get("authors", [])])
            })

        offset += batch_size
        time.sleep(0.2)  # be polite

    print(f"✅ Collected {offset} papers for '{query}'")

# Save to CSV
df = pd.DataFrame(all_papers)
df.to_csv("semantic_scholar_top10000.csv", index=False, encoding="utf-8")

# Print sample
print("\n📄 First 5 papers collected:")
print(df.head())



🔍 Collecting top 2500 papers for query: 'machine learning'...
⏳ Hit rate limit at offset 300. Sleeping for 60 seconds...
⏳ Hit rate limit at offset 400. Sleeping for 60 seconds...
⏳ Hit rate limit at offset 600. Sleeping for 60 seconds...
⏳ Hit rate limit at offset 600. Sleeping for 60 seconds...
❌ Error at offset 1000: 400
✅ Collected 1000 papers for 'machine learning'

🔍 Collecting top 2500 papers for query: 'data science'...
⏳ Hit rate limit at offset 800. Sleeping for 60 seconds...
❌ Error at offset 1000: 400
✅ Collected 1000 papers for 'data science'

🔍 Collecting top 2500 papers for query: 'artificial intelligence'...
⏳ Hit rate limit at offset 700. Sleeping for 60 seconds...
❌ Error at offset 1000: 400
✅ Collected 1000 papers for 'artificial intelligence'

🔍 Collecting top 2500 papers for query: 'information extraction'...
⏳ Hit rate limit at offset 800. Sleeping for 60 seconds...
❌ Error at offset 1000: 400
✅ Collected 1000 papers for 'information extraction'

📄 First 5 papers

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
# Write code for each of the sub parts with proper comments.



# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Your code here



# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog